In [1]:
"""Import modules required to run the Jupyter notebook."""
import os

OPENSLIDE_PATH = 'D:/NSC2024/source-code/openslide_binary/bin'
with os.add_dll_directory(OPENSLIDE_PATH):
    import openslide
    
# Clear logger to use tiatoolbox.logger
import logging
import warnings

if logging.getLogger().hasHandlers():
    logging.getLogger().handlers.clear()

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch
from matplotlib import cm

from tiatoolbox import logger
from tiatoolbox.models.architecture.unet import UNetModel
from tiatoolbox.models.engine.semantic_segmentor import (
    IOSegmentorConfig,
    SemanticSegmentor,
)
from tiatoolbox.utils.misc import download_data, imread
from tiatoolbox.utils.visualization import overlay_prediction_mask
from tiatoolbox.wsicore.wsireader import WSIReader

mpl.rcParams["figure.dpi"] = 300  # for high resolution figure in notebook
mpl.rcParams["figure.facecolor"] = "white"  # To make sure text is visible in dark mode
warnings.filterwarnings("ignore")

In [8]:
ON_GPU = False  # Should be changed to False if no cuda-enabled GPU is available

In [3]:
# These file name are used for
img_file_name = "sample_tile.jpg"
wsi_file_name = "sample_wsi.svs"
mini_wsi_file_name = "mini_wsi.svs"
model_file_name = "tissue_mask_model.pth"

logger.info("Download has started. Please wait...")

# Downloading sample image tile
download_data(
    "https://tiatoolbox.dcs.warwick.ac.uk/sample_imgs/breast_tissue.jpg",
    img_file_name,
)

# Downloading sample whole-slide image
download_data(
    "https://tiatoolbox.dcs.warwick.ac.uk/sample_wsis/wsi4_12k_12k.svs",
    wsi_file_name,
)

# Downloading mini whole-slide image
download_data(
    "https://tiatoolbox.dcs.warwick.ac.uk/sample_wsis/CMU-1.ndpi",
    mini_wsi_file_name,
)

# Download external model
download_data(
    "https://tiatoolbox.dcs.warwick.ac.uk//models/seg/fcn-tissue_mask.pth",
    model_file_name,
)

logger.info("Download is complete.")

|2024-07-19|22:57:42.899| [INFO] Download has started. Please wait...
|2024-07-19|22:58:59.987| [INFO] Download is complete.


In [12]:
# Tile prediction
bcc_segmentor = SemanticSegmentor(
    pretrained_model="fcn_resnet50_unet-bcss",
    # num_loader_workers=1,
    batch_size=1,
)

output = bcc_segmentor.predict(
    [img_file_name],
    save_dir="sample_tile_results/",
    mode="tile",
    resolution=1.0,
    units="baseline",
    patch_input_shape=[1024, 1024],
    patch_output_shape=[512, 512],
    stride_shape=[512, 512],
    on_gpu=ON_GPU,
    crash_on_exception=True,
)

|2024-07-19|23:04:03.664| [WARNING] WSIPatchDataset only reads image tile at `units="baseline"`. Resolutions will be converted to baseline value.
|2024-07-19|23:04:03.760| [WARNING] WSIPatchDataset only reads image tile at `units="baseline"`. Resolutions will be converted to baseline value.
|2024-07-19|23:04:03.915| [WARNING] Raw data is None.
|2024-07-19|23:04:03.916| [WARNING] Unknown scale (no objective_power or mpp)
Process Batch: 100%|############################| 64/64 [02:21<00:00,  2.21s/it]

|2024-07-19|23:06:27.505| [INFO] Finish: 0
|2024-07-19|23:06:27.507| [INFO] --Input: sample_tile.jpg
|2024-07-19|23:06:27.508| [INFO] --Output: D:\NSC2024\source-code\main\TIAToolbox\sample_tile_results\0


In [13]:
logger.info(
    "Prediction method output is: %s, %s",
    output[0][0],
    output[0][1],
)
tile_prediction_raw = np.load(
    output[0][1] + ".raw.0.npy",
)  # Loading the first prediction [0] based on the output address [1]
logger.info(
    "Raw prediction dimensions: (%d, %d, %d)",
    tile_prediction_raw.shape[0],
    tile_prediction_raw.shape[1],
    tile_prediction_raw.shape[2],
)

# Simple processing of the raw prediction to generate semantic segmentation task
tile_prediction = np.argmax(
    tile_prediction_raw,
    axis=-1,
)  # select the class with highest probability
logger.info(
    "Processed prediction dimensions: (%d, %d)",
    tile_prediction.shape[0],
    tile_prediction.shape[1],
)

# showing the predicted semantic segmentation
tile = imread(img_file_name)
logger.info(
    "Input image dimensions: (%d, %d, %d)",
    tile.shape[0],
    tile.shape[1],
    tile.shape[2],
)

fig = plt.figure()
label_names_dict = {
    0: "Tumour",
    1: "Stroma",
    2: "Inflamatory",
    3: "Necrosis",
    4: "Others",
}
for i in range(5):
    ax = plt.subplot(1, 5, i + 1)
    (
        plt.imshow(tile_prediction_raw[:, :, i]),
        plt.xlabel(
            label_names_dict[i],
        ),
        ax.axes.xaxis.set_ticks([]),
        ax.axes.yaxis.set_ticks([]),
    )
fig.suptitle("Row prediction maps for different classes", y=0.65)

# showing processed results
fig2 = plt.figure()
ax1 = plt.subplot(1, 2, 1), plt.imshow(tile), plt.axis("off")
ax2 = plt.subplot(1, 2, 2), plt.imshow(tile_prediction), plt.axis("off")
fig2.suptitle("Processed prediction map", y=0.82)

|2024-07-19|23:06:37.470| [INFO] Prediction method output is: sample_tile.jpg, D:\NSC2024\source-code\main\TIAToolbox\sample_tile_results\0
|2024-07-19|23:06:37.601| [INFO] Raw prediction dimensions: (4000, 4000, 5)
|2024-07-19|23:06:37.835| [INFO] Processed prediction dimensions: (4000, 4000)
|2024-07-19|23:06:37.999| [INFO] Input image dimensions: (4000, 4000, 3)


Text(0.5, 0.82, 'Processed prediction map')